# Exploratory Data Analys - Titanic

Obiettivo: comprendere il dataset e individuare le feature utili per un primo modello di classificazione.

In [1]:
import pandas as pd

## Caricamento del dataset

In [2]:
df = pd.read_csv("../data/titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Struttura del dataset

Il dataset contiene 891 osservazioni e 12 colonne.

Alcune colonne presentano valori mancanti, in particolare 'Age' e 'Cabin'.

Sono presenti sia variabili numeriche che categoriche, che richiederanno una fase di preprocessing prima dell'addestramento del modello.

In [3]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    str    
 4   Sex          891 non-null    str    
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    str    
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    str    
 11  Embarked     889 non-null    str    
dtypes: float64(2), int64(5), str(5)
memory usage: 118.9 KB


## Valori mancanti (missing values)

In [4]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Considerazione sui valori mancanti

La colonna 'Cabin' presenta un numero molto elevato di valori mancanti e viene esclusa dall'analisi.

La colonna 'Age' contiene valori mancanti ma rimane informativa e verrà gestita in fase di preprocessing.

La colonna 'Embarked' presenta pochi valori mancanti e può essere facilmente sistemata. 

Le strategie di imputazione verranno definite in una fase successiva in base al modello utilizzato.

## Gestione dei valori mancanti - Age

In [5]:
df["Age"].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

Per la colonna 'Age' viene utilizzata la **mediana** per imputare i valori mancanti, in quanto meno sensibile ai valori estremi rispetto alla media.

In [6]:
df["Age"] = df["Age"].fillna(df["Age"].median())

In [7]:
df["Age"].isnull().sum()

np.int64(0)

In [8]:
df["Embarked"].value_counts(dropna=False)

Embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64

## Gestione valori mancanti - Embarked

La colonna 'Embarked' presenta pochi valori mancanti.
Si utilizza la **moda** per l'imputazione, in quanto rappresenta il valore più frequente nel dataset.

In [9]:
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

In [10]:
df["Embarked"].isnull().sum()

np.int64(0)

## Esclusione della colonna Cabin

La colonna 'Cabin' presenta un numero molto elevato di valori mancanti (oltre il 75% del dataset) e viene quindi esclusa dall'analisi, in quanto non sufficientemente informativa.

In [11]:
df = df.drop(columns=["Cabin"], errors="ignore")

df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked'],
      dtype='str')

In [12]:
df["Sex"].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [13]:
df["Embarked"].value_counts()

Embarked
S    646
C    168
Q     77
Name: count, dtype: int64

## Encoding delle variabili categoriche

La variabile 'Sex' viene codificata tramite **label encoding**, in quanto binaria.

La variabile 'Embarked' viene codificata tramite **one-hot encoding**, in quanto categoriale senza ordine naturale.

In [14]:
df["Sex"] = df["Sex"].map({"male": 0, "female": 1})

In [15]:
df["Sex"].value_counts()

Sex
0    577
1    314
Name: count, dtype: int64

In [16]:
df = pd.get_dummies(df, columns=["Embarked"], drop_first=True)

In [17]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,False,True
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,False,False
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,False,True
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,False,True
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,False,True


In [18]:
y = df["Survived"]
X = df.drop(columns=["Survived", "PassengerId", "Name", "Ticket"])
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,3,0,22.0,1,0,7.2500,False,True
1,1,1,38.0,1,0,71.2833,False,False
2,3,1,26.0,0,0,7.9250,False,True
3,1,1,35.0,1,0,53.1000,False,True
4,3,0,35.0,0,0,8.0500,False,True


## Train / Test Split

Il dataset viene suddiviso in un insieme di **training** e uno di **test** (80% / 20%) per valutare le prestazioni del modello su dati non visti.

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((712, 8), (179, 8))

## Primo modello - Logistic Regression

Si utilizza una **Logistic Regression** come modello di baseline per la classificazione binaria del target 'Survived'.

In [20]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

model.score(X_test, y_test)

0.8100558659217877

## Valutazione del modello

Oltre all'accuracy, vengono analizzate la **confusion matrix** e le metriche di classificazione (precision, recall, f1-score).

In [21]:
y_pred = model.predict(X_test)

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm

array([[90, 15],
       [19, 55]])

In [22]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.86      0.84       105
           1       0.79      0.74      0.76        74

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179



### Considerazioni sul modello

Il modello di baseline mostra buone prestazioni complessive (accuracy ≈ 0.81).

La classe 'Survised' presenta un buon compromesso tra precision e recall, anche se il modello tende a perdere alcuni sopravvissuti (false negative).

Il risultato è coerente con una Logistic Regression non ottimizzata e conferma la correttezza del preprocessing effettuato.

## Conclusioni

In questo notebook è stata implementata una pipeline di Machine Learning per un problema di classificazione supervisionata.

Il modello di baseline ha mostrato buone prestazioni, confermando la correttezza delle scelte di preprocessing e encoding.